In [1]:
# fix the cross-validation process using Pipeline
from sklearn.pipeline import make_pipeline
from __future__ import division # ensure that all division is float division
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn import metrics
#import all the necessary modules
import os, sys, re
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
sns.set_style("whitegrid")

In [2]:
os.chdir(r"C:\Users\Dominique Njinkeu\Documents\GitHub\Projects\notebook")
df=pd.read_csv("tweets_sample.csv")
df.Support.value_counts()
# map the label values for scikit-learn
df["Label"]=df.Support.map({"Neutral":0,"Panthers":1,"Broncos":2})
#determine feature and label variables
X=df.text
y=df.Label
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [3]:
#multinomialNB classifier
from sklearn.naive_bayes import MultinomialNB
clfNB=MultinomialNB()
#knn classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
#linear SVM
from sklearn import svm
linear_svc=LinearSVC(loss="l1",C=10)
#RBF SVM
rbf_svm=svm.SVC(kernel='rbf',C=470)
#Logistic Regression
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e5)

In [4]:
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from time import time
os.chdir(r"C:\Users\Dominique Njinkeu\Documents\GitHub\Projects\codes")
import ParseOutText

In [5]:
#find knn model
from sklearn.pipeline import make_pipeline

pipe = Pipeline([
    #('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfVectorizer(stop_words="english",ngram_range=(1,2),
                              max_df=0.5,norm="l2",use_idf=False,
                             max_features=25)
     ),
    
    ('clf', KNeighborsClassifier(n_neighbors=10)),
])
td=TfidfVectorizer(stop_words="english",ngram_range=(1,2),
                              max_df=0.5,norm="l2",use_idf=False,
                             max_features=25)
knn=KNeighborsClassifier(n_neighbors=10)

ParseOutText.confusion_ROC(td,knn)

cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

Features:  25
confusion matrix 

[[369  30  12]
 [ 22 176   9]
 [ 15  14 103]]


0.85095395183734701

In [6]:
#find linear svc

pipe = Pipeline([
    #('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfVectorizer(max_features=900,ngram_range=(1,2),
                              stop_words='english',norm="l2",use_idf=False,
                             max_df=.5)
     ),
    
    ('clf', linear_svc),
])
td=TfidfVectorizer(ngram_range=(1,2),
                              stop_words='english',norm="l2",use_idf=False,
                             max_df=.5)
     
ParseOutText.confusion_ROC(td,linear_svc)

cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

Features:  17559
confusion matrix 

[[371  28  12]
 [ 13 181  13]
 [  6  13 113]]


C:\Users\Dominique Njinkeu\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\svm\classes.py:197: DeprecationWarning: loss='l1' has been deprecated in favor of loss='hinge' as of 0.16. Backward compatibility for the loss='l1' will be removed in 1.0
  DeprecationWarning)


0.86664731479865065

In [7]:
#find log reg
pipe = Pipeline([
    #('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfVectorizer(max_features=190,ngram_range=(1,2),
                              stop_words='english',norm="l2",use_idf=False,
                             max_df=.5)
     ),
    
    ('clf', logreg),
])
td=TfidfVectorizer(max_features=190,ngram_range=(1,2),
                              stop_words='english',norm="l2",use_idf=False,
                             max_df=.5)
     
ParseOutText.confusion_ROC(td,linear_svc)

cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

Features:  190
confusion matrix 

[[368  29  14]
 [ 14 180  13]
 [  9  10 113]]


0.85232280400928162

In [18]:
#find log reg
pipe = Pipeline([
    #('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfVectorizer(ngram_range=(1,3),stop_words="english",max_features=190,
                             )
     ),
    
    ('clf', logreg),
])
td=TfidfVectorizer(ngram_range=(1,3),stop_words="english",max_features=190)
     
     
ParseOutText.confusion_ROC(td,logreg)

cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

Features:  190
confusion matrix 

[[375  24  12]
 [ 21 172  14]
 [ 15  14 103]]


0.84433606326672328

In [9]:
clf=svm.SVC(kernel='linear')
X_train_dtm=td.fit_transform(X_train)
print "Features: ",X_train_dtm.shape[1]
clf.fit(X_train_dtm,y_train)
X_test_dtm=td.transform(X_test)
y_pred_class=clf.predict(X_test_dtm)

Features:  190


In [19]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, classification_report
from pprint import pprint
print(classification_report(y_test,y_pred_class))

             precision    recall  f1-score   support

          0       0.89      0.89      0.89       411
          1       0.81      0.88      0.84       207
          2       0.81      0.72      0.76       132

avg / total       0.86      0.86      0.86       750



In [25]:
def classifier(vect,clf):
	scores=[]
	X_train_dtm=vect.fit_transform(X_train)
	print "Features: ",X_train_dtm.shape[1]
	clf.fit(X_train_dtm,y_train)
	X_test_dtm=vect.transform(X_test)
	y_pred_class=clf.predict(X_test_dtm)
	print "classification_report"
	print classification_report(y_test,y_pred_class)
	accuracy=accuracy_score(y_test,y_pred_class)
	f1=f1_score(y_test,y_pred_class,average=None)
	precision=precision_score(y_test,y_pred_class,average=None)
	recall=recall_score(y_test,y_pred_class,average=None)
	print "accuracy_score",accuracy
	for prec,rec,f in zip(precision,recall,f1):
		scores.append(prec)
		scores.append(rec)
		scores.append(f)
	return scores
classifier(td,linear_svc)
scores=[]

Features:  190
classification_report
             precision    recall  f1-score   support

          0       0.91      0.87      0.89       411
          1       0.81      0.88      0.84       207
          2       0.78      0.77      0.77       132

avg / total       0.86      0.86      0.86       750

accuracy_score 0.857333333333


[0.91116751269035534,
 0.87347931873479323,
 0.89192546583850929,
 0.80616740088105732,
 0.88405797101449279,
 0.84331797235023043,
 0.78294573643410847,
 0.76515151515151514,
 0.77394636015325668]

In [11]:
mbn_params = {'alpha': [a*0.1 for a in range(0,11)]}
mbn_clf = GridSearchCV(MultinomialNB(), mbn_params, cv=10)
mbn_clf.fit(X_train_dtm, y_train)
print 'best parameters'
print mbn_clf.best_params_

best parameters
{'alpha': 0.0}


In [12]:
"""Best score: 0.856
Best parameters set for tfidf:
        clf__algorithm: 'ball_tree'
        clf__leaf_size: 15
        clf__p: 2
        clf__weights: 'distance'
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.7min finished
"""

"Best score: 0.856\nBest parameters set for tfidf:\n        clf__algorithm: 'ball_tree'\n        clf__leaf_size: 15\n        clf__p: 2\n        clf__weights: 'distance'\n[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.7s\n[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.7min finished\n"

In [13]:
lsvm_params = {'C':[1,10,100,1000], 'loss':['l1', 'l2']}
lsvm_clf = GridSearchCV(LinearSVC(), lsvm_params, cv=5)
lsvm_clf.fit(X_train_dtm, y_train)
print 'best parameters'
print lsvm_clf.best_params_

C:\Users\Dominique Njinkeu\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\svm\classes.py:197: DeprecationWarning: loss='l1' has been deprecated in favor of loss='hinge' as of 0.16. Backward compatibility for the loss='l1' will be removed in 1.0
  DeprecationWarning)
C:\Users\Dominique Njinkeu\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\svm\classes.py:197: DeprecationWarning: loss='l1' has been deprecated in favor of loss='hinge' as of 0.16. Backward compatibility for the loss='l1' will be removed in 1.0
  DeprecationWarning)
C:\Users\Dominique Njinkeu\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\svm\classes.py:197: DeprecationWarning: loss='l1' has been deprecated in favor of loss='hinge' as of 0.16. Backward compatibility for the loss='l1' will be removed in 1.0
  DeprecationWarning)
C:\Users\Dominique Njinkeu\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\svm\classes.py:197: DeprecationWarning: loss='l1' has been deprec

best parameters
{'loss': 'l1', 'C': 1}


C:\Users\Dominique Njinkeu\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\svm\classes.py:197: DeprecationWarning: loss='l1' has been deprecated in favor of loss='hinge' as of 0.16. Backward compatibility for the loss='l1' will be removed in 1.0
  DeprecationWarning)


In [14]:
log_params={"C":[0.001, 0.01, 0.1, 1, 10, 100, 1000,10000],"penalty":["l1","l2"]}
log_clf = GridSearchCV(linear_model.LogisticRegression(), log_params, cv=5)
log_clf.fit(X_train_dtm, y_train)
print 'best parameters'
print log_clf.best_params_

best parameters
{'penalty': 'l1', 'C': 10}


In [15]:
#find knn model
from sklearn.pipeline import make_pipeline

pipe = Pipeline([
    
    ('tfidf', TfidfVectorizer(stop_words="english",ngram_range=(1,2),
                              max_df=0.5,norm="l2",use_idf=False,
                             max_features=25)
     ),
    
    ('clf', KNeighborsClassifier(n_neighbors=10,algorithm="ball_tree",
                                leaf_size=15,p=2,weights="distance")),
])
td=TfidfVectorizer(stop_words="english",ngram_range=(1,2),
                              max_df=0.5,norm="l2",use_idf=False,
                             max_features=25)
knn=KNeighborsClassifier(n_neighbors=10,algorithm="ball_tree",
                                leaf_size=15,p=2,weights="distance")

ParseOutText.confusion_ROC(td,knn)

cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

Features:  25
confusion matrix 

[[372  27  12]
 [ 24 174   9]
 [ 15  12 105]]


C:\Users\Dominique Njinkeu\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\neighbors\base.py:229: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


0.85229175929446688

ImportError: No module named brew.base